In [1]:
%load_ext autoreload
%autoreload 1
import ROOT
import cppyy
from DataFormats.FWLite import Handle
import sys
sys.path.append("..")
#import edmReader.pythonizations
%aimport edmReader.edmReader
from edmReader.edmReader import EdmReader

Welcome to JupyROOT 6.26/11


In [2]:
evt_baseline = EdmReader("/workspaces/repo/runs_results/diffEgamma/baseline/step3.root")
evt_new = EdmReader("/workspaces/repo/runs_results/diffEgamma/superclsEgamma-ce99bdc0723-v3/step3.root")
events_baseline_iter = iter(evt_baseline)
events_new_iter = iter(evt_new)
evt_baseline._createFWLiteEvent()
evt_new._createFWLiteEvent()

'chain'

cling::DynamicLibraryManager::loadLibrary(): libcuda.so.1: cannot open shared object file: No such file or directory
Error in <TInterpreter::TCling::AutoLoad>: failure loading library libSimTrackerTrackTriggerAssociation.so for map<string,ULong64_t>
cling::DynamicLibraryManager::loadLibrary(): libcuda.so.1: cannot open shared object file: No such file or directory
Error in <TInterpreter::TCling::AutoLoad>: failure loading library libSimTrackerTrackTriggerAssociation.so for map<edm::Ref<edmNew::DetSetVector<TTCluster<edm::Ref<edm::DetSetVector<Phase2TrackerDigi>,Phase2TrackerDigi,edm::refhelper::FindForDetSetVector<Phase2TrackerDigi> > > >,TTCluster<edm::Ref<edm::DetSetVector<Phase2TrackerDigi>,Phase2TrackerDigi,edm::refhelper::FindForDetSetVector<Phase2TrackerDigi> > >,edmNew::DetSetVector<TTCluster<edm::Ref<edm::DetSetVector<Phase2TrackerDigi>,Phase2TrackerDigi,edm::refhelper::FindForDetSetVector<Phase2TrackerDigi> > > >::FindForDetSetVector>,vector<edm::Ptr<TrackingParticle> > >
clin

In [25]:
while True:
    next(events_baseline_iter)
    #next(events_new_iter)
    #evt.getByLabel("genParticles", genparts_h)
    genparts = evt_baseline.get("vector<reco::GenParticle>", "genParticles")
    if genparts[1].eta() > 1.6 and genparts[1].eta() < 2.8:
        evt_new.object().to(evt_baseline.object().id())
        if evt_new.ticlEgammaSuperClusters.size() > 0:
            break

In [26]:
genparts

std::vector<reco::GenParticle>[reco::GenParticle(energy=15000.0,charge=0,eta=0.0,pdgId=990,isHardProcess=False),reco::GenParticle(energy=185.68380511322417,charge=-1,eta=2.2958824634552,pdgId=11,isHardProcess=False),reco::GenParticle(energy=185.68380511322417,charge=1,eta=-2.2958824634552,pdgId=-11,isHardProcess=False)]

In [27]:
evt_baseline.oldHgcalSuperclusters[0].phiWidth()

0.005269800778478384

In [28]:
evt_new.ticlEgammaSuperClusters[2]

reco::SuperCluster(energy=172.51873779296875,rawEnergy=183.9346160888672,correctedEnergy=172.51873779296875,clustersSize=1,eta=2.2839144363339106,algo=1000,caloID=CaloID: 0,seed=<cppyy.gbl.edm.Ptr<reco::CaloCluster> object at 0x392c0620>)

In [29]:
list(evt_new.ticlEgammaSuperClusters[2].clusters())[0]#.get().hitsAndFractions()

reco::CaloCluster(energy=183.9346160888672,correctedEnergy=183.9346160888672,eta=2.2839144363339106,algo=5,caloID=CaloID: 1024,seed=DetId(det=HGCalEE,subdetId=0)

In [30]:
evt_baseline.electronSeeds

std::vector<reco::ElectronSeed>[ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=186.3620520702587,CaloClusterEta=2.2843319397621134),ElectronSeed(CaloDriven

In [31]:
evt_new.electronSeeds

std::vector<reco::ElectronSeed>[ElectronSeed(CaloDriven,charge=1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSeed(CaloDriven,charge=-1,CaloClusterEnergy=168.45266723632812,CaloClusterEta=-2.3093475126443925),ElectronSee

In [10]:
evt_baseline.get("vector<reco::GsfTrack>","electronGsfTracks" )

std::vector<reco::GsfTrack>[reco::GsfTrack(etaMode=-2.496912847005952,ptMode=10.612227166083121,qoverpMode=0.015413225981225324,outerEta=-2.49689123525632,outerPt=6.474755515014397,seedDirection=1),reco::GsfTrack(etaMode=2.496627919935485,ptMode=11.274840500713871,qoverpMode=-0.0145114814889851,outerEta=2.4963226662276305,outerPt=2.946369320300965,seedDirection=1)]

In [11]:
evt_new.get("vector<reco::GsfTrack>","electronGsfTracks" )

std::vector<reco::GsfTrack>[reco::GsfTrack(etaMode=-2.496912847005952,ptMode=10.612227166083121,qoverpMode=0.015413225981225324,outerEta=-2.49689123525632,outerPt=6.474755515014397,seedDirection=1),reco::GsfTrack(etaMode=2.496627919935485,ptMode=11.274840500713871,qoverpMode=-0.0145114814889851,outerEta=2.4963226662276305,outerPt=2.946369320300965,seedDirection=1)]

In [12]:
evt_baseline.printPfBlocks(filterOutNonEndcap=True)

In [13]:
list(evt_new.pfBlocks[67].elements())[-1].clusterRef().isAvailable()

True

In [14]:
evt_new.printPfBlocks(filterOutNonEndcap=True)

In [15]:
evt_new.get("vector<reco::GsfTrack>","electronGsfTracks" )[0].__dir__()

['__lifeline',
 '__module__',
 '__doc__',
 '__assign__',
 '__init__',
 'chargeMode',
 'covarianceMode',
 'errorMode',
 'etaMode',
 'etaModeError',
 'gsfExtra',
 'lambdaMode',
 'lambdaModeError',
 'momentumMode',
 'pMode',
 'parameterMode',
 'parametersMode',
 'phiMode',
 'phiModeError',
 'ptMode',
 'ptModeError',
 'pxMode',
 'pyMode',
 'pzMode',
 'qoverpMode',
 'qoverpModeError',
 'setGsfExtra',
 'setMode',
 'thetaMode',
 'thetaModeError',
 'dimensionMode',
 'covarianceSizeMode',
 '__repr__',
 'extra',
 'fillInner',
 'fillOuter',
 'found',
 'innerDetId',
 'innerMomentum',
 'innerOk',
 'innerPosition',
 'innerStateCovariance',
 'lost',
 'outerDetId',
 'outerEta',
 'outerMomentum',
 'outerOk',
 'outerP',
 'outerPhi',
 'outerPosition',
 'outerPt',
 'outerPx',
 'outerPy',
 'outerPz',
 'outerRadius',
 'outerStateCovariance',
 'outerTheta',
 'outerX',
 'outerY',
 'outerZ',
 'recHit',
 'recHits',
 'recHitsBegin',
 'recHitsEnd',
 'recHitsSize',
 'residuals',
 'seedDirection',
 'seedRef',
 'set

In [16]:
evt_baseline.get("vector<reco::ElectronSeed>","electronMergedSeeds" )
evt_baseline.get("vector<reco::GsfTrack>","electronGsfTracks" )

std::vector<reco::GsfTrack>[reco::GsfTrack(etaMode=-2.496912847005952,ptMode=10.612227166083121,qoverpMode=0.015413225981225324,outerEta=-2.49689123525632,outerPt=6.474755515014397,seedDirection=1),reco::GsfTrack(etaMode=2.496627919935485,ptMode=11.274840500713871,qoverpMode=-0.0145114814889851,outerEta=2.4963226662276305,outerPt=2.946369320300965,seedDirection=1)]